<a href="https://colab.research.google.com/github/ithalo-teodosio/pratica_ML/blob/main/ML_Q8_Hackaton_Recomendacao_filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instituição: Unifacisa
# Autor: Ithalo Teodósio Nascimento
# Q8: Recomendação de filmes com filtragem colaborativa

**Questão 8 (avançado) - Recomendação de Filmes com Filtragem Colaborativa**

Uma plataforma de streaming deseja sugerir filmes para os usuários com base nas avaliações de outros usuários.

Tarefas:

- Utilize um dataset de avaliações de filmes (exemplo: MovieLens).
- Implemente um modelo de filtragem colaborativa baseado em usuários e itens.
- Compare a filtragem colaborativa com abordagens baseadas em aprendizado profundo (exemplo: Autoencoders).
- Avalie o desempenho com métricas como RMSE e MAE.

Pergunta: Qual abordagem foi mais eficiente na recomendação de filmes? Como melhorar o sistema de recomendação?

In [1]:
# Instalar a biblioteca do TensorFlow
!pip install tensorflow --quiet

# Bibliotecas
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense


In [2]:
# Carregando direto da web
url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
colunas = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv(url, sep="\t", names=colunas)
df.drop('timestamp', axis=1, inplace=True)

df.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [3]:
# Criando matriz de usuários x itens
matriz = df.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)

# Calculando a similaridade entre usuários
similaridade = cosine_similarity(matriz)

# Função simples para prever nota com base em usuários parecidos
def prever(user_id, item_id):
    notas_item = matriz[item_id]
    similaridades = similaridade[user_id - 1]  # IDs começam em 1
    mascara = notas_item > 0
    if mascara.sum() == 0:
        return matriz.loc[user_id].mean()
    notas = notas_item[mascara]
    sims = similaridades[mascara]
    return np.dot(notas, sims) / sims.sum()

In [5]:
# Separando dados em treino e teste
treino, teste = train_test_split(df, test_size=0.2, random_state=42)

# Prevendo para o teste
teste = teste.copy()
teste['previsao'] = teste.apply(lambda row: prever(row['user_id'], row['item_id']), axis=1)

# Erros
rmse_cf = np.sqrt(mean_squared_error(teste['rating'], teste['previsao']))
mae_cf = mean_absolute_error(teste['rating'], teste['previsao'])

print(f"📈 CF → RMSE: {rmse_cf:.2f} | MAE: {mae_cf:.2f}")

📈 CF → RMSE: 0.95 | MAE: 0.76


In [6]:
# Pegando os dados da matriz de usuário x item
X = matriz.values

# Autoencoder básico: entrada → camada escondida → saída
entrada = Input(shape=(X.shape[1],))
escondida = Dense(64, activation='relu')(entrada)
saida = Dense(X.shape[1], activation='linear')(escondida)

modelo = Model(entrada, saida)
modelo.compile(optimizer='adam', loss='mse')

# Treinando (usando os próprios dados como entrada e saída)
modelo.fit(X, X, epochs=10, batch_size=64, verbose=0)

In [8]:
# Previsões
preds = modelo.predict(X)

# Avaliando só onde havia nota (para não considerar os zeros)
mascara = X > 0
rmse_ae = np.sqrt(mean_squared_error(X[mascara], preds[mascara]))
mae_ae = mean_absolute_error(X[mascara], preds[mascara])

print(f"🧠 Autoencoder → RMSE: {rmse_ae:.2f} | MAE: {mae_ae:.2f}")

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
🧠 Autoencoder → RMSE: 2.22 | MAE: 1.91


In [9]:
print("📊 Comparando os modelos:")
print(f"Filtragem Colaborativa → RMSE: {rmse_cf:.2f} | MAE: {mae_cf:.2f}")
print(f"Autoencoder            → RMSE: {rmse_ae:.2f} | MAE: {mae_ae:.2f}")

📊 Comparando os modelos:
Filtragem Colaborativa → RMSE: 0.95 | MAE: 0.76
Autoencoder            → RMSE: 2.22 | MAE: 1.91


**Conclusão:**

**Filtragem Colaborativa** é simples, rápida e funciona bem com dados menores.

**Autoencoder** aprende padrões mais profundos, mas precisa de mais dados e processamento.

Para melhorar:

- Use mais dados (ex: MovieLens 1M)

- Adicione informações extras (gênero, idade, tipo de filme)

- Teste outros modelos como Matrix Factorization ou Redes Neurais mais profundas.